# Training and Deploying a Custom Image Classifier with Amazon SageMaker

## Configure where to fetch our training data

All of our images live inside an S3 bucket, organized into folders in a structure similar to this:

```
my_training_classes
├── person
│   ├── han.jpg
│   ├── leia.jpg
|   ├── luke.jpg
│   └── . . .
└── ship
│   ├── millenium_falcon.jpg
│   ├── tie-fighter.jpg    
│   ├── x-wing.jpg
│   ├── . . .
└── . . .
```

In [1]:
# An S3 Bucket Name
data_bucket_name='concab-asl-west'

# A prefix name inside the S3 bucket containing sub-folders of images (one per label class)
dataset_name = 'image-classification-pics' 

## Setting up the environment
Here we set up the linkage and authentication to AWS services

- The role used to give learning and hosting access to your data. This will automatically be obtained from the role used to start the notebook
- A `session` variable that holds some configuration state for interacting with SageMaker from Python and contains some methods for preparing input data
- A reference to the Amazon sagemaker image classification docker image 

More info about the SageMaker built-in Image Classification algorithm here: https://docs.aws.amazon.com/sagemaker/latest/dg/image-classification.html

In [10]:
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = "arn:aws:iam::501088278124:role/service-role/AmazonSageMaker-ExecutionRole-20190610T112753"
sess = sagemaker.Session()

training_image = get_image_uri(sess.boto_region_name, 'image-classification', repo_version="latest")

Couldn't call 'get_role' to get Role ARN from role name FastaiSageMakerStack0-SageMakerIamRole-1CPMK6NI6VF9J to get Role path.


In [40]:
print(training_image)

433757028032.dkr.ecr.us-west-2.amazonaws.com/image-classification:latest


In [11]:
iam = boto3.resource('iam')
role = iam.Role('AmazonSageMaker-ExecutionRole-20190410T133202')


In [19]:
!aws s3 sync s3://concab-asl-west/ .
!unzip image-classification-pics.zip -d /tmp/

Archive:  image-classification-pics.zip
   creating: /tmp/image-classification-pics/a/
  inflating: /tmp/image-classification-pics/a/a0.jpg  
  inflating: /tmp/image-classification-pics/a/a1.jpg  
  inflating: /tmp/image-classification-pics/a/a10.jpg  
  inflating: /tmp/image-classification-pics/a/a100.jpg  
  inflating: /tmp/image-classification-pics/a/a101.jpg  
  inflating: /tmp/image-classification-pics/a/a102.jpg  
  inflating: /tmp/image-classification-pics/a/a103.jpg  
  inflating: /tmp/image-classification-pics/a/a104.jpg  
  inflating: /tmp/image-classification-pics/a/a105.jpg  
  inflating: /tmp/image-classification-pics/a/a106.jpg  
  inflating: /tmp/image-classification-pics/a/a107.jpg  
  inflating: /tmp/image-classification-pics/a/a108.jpg  
  inflating: /tmp/image-classification-pics/a/a109.jpg  
  inflating: /tmp/image-classification-pics/a/a11.jpg  
  inflating: /tmp/image-classification-pics/a/a110.jpg  
  inflating: /tmp/image-classification-pics/a/a111.jpg  
  infla

  inflating: /tmp/image-classification-pics/a/a244.jpg  
  inflating: /tmp/image-classification-pics/a/a245.jpg  
  inflating: /tmp/image-classification-pics/a/a246.jpg  
  inflating: /tmp/image-classification-pics/a/a247.jpg  
  inflating: /tmp/image-classification-pics/a/a248.jpg  
  inflating: /tmp/image-classification-pics/a/a249.jpg  
  inflating: /tmp/image-classification-pics/a/a25.jpg  
  inflating: /tmp/image-classification-pics/a/a250.jpg  
  inflating: /tmp/image-classification-pics/a/a251.jpg  
  inflating: /tmp/image-classification-pics/a/a252.jpg  
  inflating: /tmp/image-classification-pics/a/a253.jpg  
  inflating: /tmp/image-classification-pics/a/a254.jpg  
  inflating: /tmp/image-classification-pics/a/a255.jpg  
  inflating: /tmp/image-classification-pics/a/a256.jpg  
  inflating: /tmp/image-classification-pics/a/a257.jpg  
  inflating: /tmp/image-classification-pics/a/a258.jpg  
  inflating: /tmp/image-classification-pics/a/a259.jpg  
  inflating: /tmp/image-classifi

  inflating: /tmp/image-classification-pics/b/b128.jpg  
  inflating: /tmp/image-classification-pics/b/b129.jpg  
  inflating: /tmp/image-classification-pics/b/b13.jpg  
  inflating: /tmp/image-classification-pics/b/b130.jpg  
  inflating: /tmp/image-classification-pics/b/b131.jpg  
  inflating: /tmp/image-classification-pics/b/b132.jpg  
  inflating: /tmp/image-classification-pics/b/b133.jpg  
  inflating: /tmp/image-classification-pics/b/b134.jpg  
  inflating: /tmp/image-classification-pics/b/b135.jpg  
  inflating: /tmp/image-classification-pics/b/b136.jpg  
  inflating: /tmp/image-classification-pics/b/b137.jpg  
  inflating: /tmp/image-classification-pics/b/b138.jpg  
  inflating: /tmp/image-classification-pics/b/b139.jpg  
  inflating: /tmp/image-classification-pics/b/b14.jpg  
  inflating: /tmp/image-classification-pics/b/b140.jpg  
  inflating: /tmp/image-classification-pics/b/b141.jpg  
  inflating: /tmp/image-classification-pics/b/b142.jpg  
  inflating: /tmp/image-classific

  inflating: /tmp/image-classification-pics/b/b289.jpg  
  inflating: /tmp/image-classification-pics/b/b29.jpg  
  inflating: /tmp/image-classification-pics/b/b290.jpg  
  inflating: /tmp/image-classification-pics/b/b291.jpg  
  inflating: /tmp/image-classification-pics/b/b292.jpg  
  inflating: /tmp/image-classification-pics/b/b293.jpg  
  inflating: /tmp/image-classification-pics/b/b294.jpg  
  inflating: /tmp/image-classification-pics/b/b295.jpg  
  inflating: /tmp/image-classification-pics/b/b296.jpg  
  inflating: /tmp/image-classification-pics/b/b297.jpg  
  inflating: /tmp/image-classification-pics/b/b298.jpg  
  inflating: /tmp/image-classification-pics/b/b299.jpg  
  inflating: /tmp/image-classification-pics/b/b3.jpg  
  inflating: /tmp/image-classification-pics/b/b30.jpg  
  inflating: /tmp/image-classification-pics/b/b300.jpg  
  inflating: /tmp/image-classification-pics/b/b301.jpg  
  inflating: /tmp/image-classification-pics/b/b302.jpg  
  inflating: /tmp/image-classificat

  inflating: /tmp/image-classification-pics/c/c177.jpg  
  inflating: /tmp/image-classification-pics/c/c178.jpg  
  inflating: /tmp/image-classification-pics/c/c179.jpg  
  inflating: /tmp/image-classification-pics/c/c18.jpg  
  inflating: /tmp/image-classification-pics/c/c180.jpg  
  inflating: /tmp/image-classification-pics/c/c181.jpg  
  inflating: /tmp/image-classification-pics/c/c182.jpg  
  inflating: /tmp/image-classification-pics/c/c183.jpg  
  inflating: /tmp/image-classification-pics/c/c184.jpg  
  inflating: /tmp/image-classification-pics/c/c185.jpg  
  inflating: /tmp/image-classification-pics/c/c186.jpg  
  inflating: /tmp/image-classification-pics/c/c187.jpg  
  inflating: /tmp/image-classification-pics/c/c188.jpg  
  inflating: /tmp/image-classification-pics/c/c189.jpg  
  inflating: /tmp/image-classification-pics/c/c19.jpg  
  inflating: /tmp/image-classification-pics/c/c190.jpg  
  inflating: /tmp/image-classification-pics/c/c191.jpg  
  inflating: /tmp/image-classific

...

  inflating: /tmp/image-classification-pics/f/f178.jpg  
  inflating: /tmp/image-classification-pics/f/f179.jpg  
  inflating: /tmp/image-classification-pics/f/f18.jpg  
  inflating: /tmp/image-classification-pics/f/f180.jpg  
  inflating: /tmp/image-classification-pics/f/f181.jpg  
  inflating: /tmp/image-classification-pics/f/f182.jpg  
  inflating: /tmp/image-classification-pics/f/f183.jpg  
  inflating: /tmp/image-classification-pics/f/f184.jpg  
  inflating: /tmp/image-classification-pics/f/f185.jpg  
  inflating: /tmp/image-classification-pics/f/f186.jpg  
  inflating: /tmp/image-classification-pics/f/f187.jpg  
  inflating: /tmp/image-classification-pics/f/f188.jpg  
  inflating: /tmp/image-classification-pics/f/f189.jpg  
  inflating: /tmp/image-classification-pics/f/f19.jpg  
  inflating: /tmp/image-classification-pics/f/f190.jpg  
  inflating: /tmp/image-classification-pics/f/f191.jpg  
  inflating: /tmp/image-classification-pics/f/f192.jpg  
  inflating: /tmp/image-classific

  inflating: /tmp/image-classification-pics/f/f52.jpg  
  inflating: /tmp/image-classification-pics/f/f53.jpg  
  inflating: /tmp/image-classification-pics/f/f54.jpg  
  inflating: /tmp/image-classification-pics/f/f55.jpg  
  inflating: /tmp/image-classification-pics/f/f56.jpg  
  inflating: /tmp/image-classification-pics/f/f57.jpg  
  inflating: /tmp/image-classification-pics/f/f58.jpg  
  inflating: /tmp/image-classification-pics/f/f59.jpg  
  inflating: /tmp/image-classification-pics/f/f6.jpg  
  inflating: /tmp/image-classification-pics/f/f60.jpg  
  inflating: /tmp/image-classification-pics/f/f61.jpg  
  inflating: /tmp/image-classification-pics/f/f62.jpg  
  inflating: /tmp/image-classification-pics/f/f63.jpg  
  inflating: /tmp/image-classification-pics/f/f64.jpg  
  inflating: /tmp/image-classification-pics/f/f65.jpg  
  inflating: /tmp/image-classification-pics/f/f66.jpg  
  inflating: /tmp/image-classification-pics/f/f67.jpg  
  inflating: /tmp/image-classification-pics/f/f68

  inflating: /tmp/image-classification-pics/g/g235.jpg  
  inflating: /tmp/image-classification-pics/g/g236.jpg  
  inflating: /tmp/image-classification-pics/g/g237.jpg  
  inflating: /tmp/image-classification-pics/g/g238.jpg  
  inflating: /tmp/image-classification-pics/g/g239.jpg  
  inflating: /tmp/image-classification-pics/g/g24.jpg  
  inflating: /tmp/image-classification-pics/g/g240.jpg  
  inflating: /tmp/image-classification-pics/g/g241.jpg  
  inflating: /tmp/image-classification-pics/g/g242.jpg  
  inflating: /tmp/image-classification-pics/g/g243.jpg  
  inflating: /tmp/image-classification-pics/g/g244.jpg  
  inflating: /tmp/image-classification-pics/g/g245.jpg  
  inflating: /tmp/image-classification-pics/g/g246.jpg  
  inflating: /tmp/image-classification-pics/g/g247.jpg  
  inflating: /tmp/image-classification-pics/g/g248.jpg  
  inflating: /tmp/image-classification-pics/g/g249.jpg  
  inflating: /tmp/image-classification-pics/g/g25.jpg  
  inflating: /tmp/image-classific

  inflating: /tmp/image-classification-pics/h/h131.jpg  
  inflating: /tmp/image-classification-pics/h/h132.jpg  
  inflating: /tmp/image-classification-pics/h/h133.jpg  
  inflating: /tmp/image-classification-pics/h/h134.jpg  
  inflating: /tmp/image-classification-pics/h/h135.jpg  
  inflating: /tmp/image-classification-pics/h/h136.jpg  
  inflating: /tmp/image-classification-pics/h/h137.jpg  
  inflating: /tmp/image-classification-pics/h/h138.jpg  
  inflating: /tmp/image-classification-pics/h/h139.jpg  
  inflating: /tmp/image-classification-pics/h/h14.jpg  
  inflating: /tmp/image-classification-pics/h/h140.jpg  
  inflating: /tmp/image-classification-pics/h/h141.jpg  
  inflating: /tmp/image-classification-pics/h/h142.jpg  
  inflating: /tmp/image-classification-pics/h/h143.jpg  
  inflating: /tmp/image-classification-pics/h/h144.jpg  
  inflating: /tmp/image-classification-pics/h/h145.jpg  
  inflating: /tmp/image-classification-pics/h/h146.jpg  
  inflating: /tmp/image-classifi

  inflating: /tmp/image-classification-pics/h/h303.jpg  
  inflating: /tmp/image-classification-pics/h/h304.jpg  
  inflating: /tmp/image-classification-pics/h/h305.jpg  
  inflating: /tmp/image-classification-pics/h/h306.jpg  
  inflating: /tmp/image-classification-pics/h/h307.jpg  
  inflating: /tmp/image-classification-pics/h/h31.jpg  
  inflating: /tmp/image-classification-pics/h/h32.jpg  
  inflating: /tmp/image-classification-pics/h/h33.jpg  
  inflating: /tmp/image-classification-pics/h/h34.jpg  
  inflating: /tmp/image-classification-pics/h/h35.jpg  
  inflating: /tmp/image-classification-pics/h/h36.jpg  
  inflating: /tmp/image-classification-pics/h/h37.jpg  
  inflating: /tmp/image-classification-pics/h/h38.jpg  
  inflating: /tmp/image-classification-pics/h/h39.jpg  
  inflating: /tmp/image-classification-pics/h/h4.jpg  
  inflating: /tmp/image-classification-pics/h/h40.jpg  
  inflating: /tmp/image-classification-pics/h/h41.jpg  
  inflating: /tmp/image-classification-pics/

## Preparing data for our model
Before we can train our model, we need to:

- Create some files that will teach SageMaker about the images in each of our classes
- Upload these additional files to S3
- Configure our model to use these files for training and validating

### Find the im2rec.py script on this system
The SageMaker image classifier algorithm needs to know about which images belong to which classes. We provide this data using either LST or RecordIO files. We'll use a python script called `im2rec.py` to create these files.

More info here: https://docs.aws.amazon.com/sagemaker/latest/dg/image-classification.html#IC-inputoutput

In [20]:
# Find im2rec in our environment and set up some other vars in our environemnt

base_dir='/tmp'

%env BASE_DIR=$base_dir
%env S3_DATA_BUCKET_NAME = $data_bucket_name
%env DATASET_NAME = $dataset_name

import sys,os

suffix='/mxnet/tools/im2rec.py'
im2rec = list(filter( (lambda x: os.path.isfile(x + suffix )), sys.path))[0] + suffix
%env IM2REC=$im2rec

env: BASE_DIR=/tmp
env: S3_DATA_BUCKET_NAME=concab-asl-west
env: DATASET_NAME=image-classification-pics
env: IM2REC=/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/tools/im2rec.py


### Get our training images from S3
In order to create training and validation RecordIO files, we need to download our images to our local filesystem.

In [21]:
# Pull our images from S3
!aws s3 sync s3://$S3_DATA_BUCKET_NAME/$DATASET_NAME $BASE_DIR/$DATASET_NAME --quiet

### Create RecordIO files from our training images
The `im2rec.py` script can create LST files and/or RecordIO files from our training data. 

More info here: https://mxnet.incubator.apache.org/versions/master/faq/recordio.html

In [22]:
%%bash
# Use the IM2REC script to convert our images into RecordIO files

# Clean up our working dir of existing LST and REC files
cd $BASE_DIR
rm *.rec
rm *.lst

# First we need to create two LST files (training and test lists), noting the correct label class for each image
# We'll also save the output of the LST files command, since it includes a list of all of our label classes
echo "Creating LST files"
python $IM2REC --list --recursive --pass-through --test-ratio=0.3 --train-ratio=0.7 $DATASET_NAME $DATASET_NAME > ${DATASET_NAME}_classes

echo "Label classes:"
cat ${DATASET_NAME}_classes

# Then we create RecordIO files from the LST files
echo "Creating RecordIO files"
python $IM2REC --num-thread=4 ${DATASET_NAME}_train.lst $DATASET_NAME
python $IM2REC --num-thread=4 ${DATASET_NAME}_test.lst $DATASET_NAME
ls -lh *.rec

Creating LST files
Label classes:
a 0
b 1
c 2
d 3
e 4
f 5
g 6
h 7
Creating RecordIO files
Creating .rec file from /tmp/image-classification-pics_train.lst in /tmp
time: 0.052478790283203125  count: 0
time: 9.417885541915894  count: 1000
Creating .rec file from /tmp/image-classification-pics_test.lst in /tmp
time: 0.05517077445983887  count: 0
-rw-rw-r-- 1 ec2-user ec2-user 25M Jul 25 17:47 image-classification-pics_test.rec
-rw-rw-r-- 1 ec2-user ec2-user 56M Jul 25 17:46 image-classification-pics_train.rec


### Upload our training and test data RecordIO files so we can train with them
Now that we have our training and test .rec files, we upload them to S3 so SageMaker can use them for training

In [23]:
# Upload our train and test RecordIO files to S3 in the bucket that our sagemaker session is using
bucket = sess.default_bucket()

s3train_path = 's3://{}/{}/train/'.format(bucket, dataset_name)
s3validation_path = 's3://{}/{}/validation/'.format(bucket, dataset_name)

# Clean up any existing data
!aws s3 rm s3://{bucket}/{dataset_name}/train --recursive
!aws s3 rm s3://{bucket}/{dataset_name}/validation --recursive

# Upload the rec files to the train and validation channels
!aws s3 cp /tmp/{dataset_name}_train.rec $s3train_path
!aws s3 cp /tmp/{dataset_name}_test.rec $s3validation_path

delete: s3://sagemaker-us-west-2-501088278124/image-classification-pics/train/image-classification-pics_train.rec
delete: s3://sagemaker-us-west-2-501088278124/image-classification-pics/validation/image-classification-pics_test.rec
upload: ../../../../tmp/image-classification-pics_train.rec to s3://sagemaker-us-west-2-501088278124/image-classification-pics/train/image-classification-pics_train.rec
upload: ../../../../tmp/image-classification-pics_test.rec to s3://sagemaker-us-west-2-501088278124/image-classification-pics/validation/image-classification-pics_test.rec


### Configure the data for our model training to use
Finally, we tell SageMaker where to find these RecordIO files to use for training

In [24]:
train_data = sagemaker.session.s3_input(
    s3train_path, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

validation_data = sagemaker.session.s3_input(
    s3validation_path, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

data_channels = {'train': train_data, 'validation': validation_data}

## Training
Now it's time to train our model!

### Create an image classifier object with some base configuration
More info here: https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.Estimator

In [35]:
s3_output_location = 's3://{}/{}/output'.format(bucket, dataset_name)

image_classifier = sagemaker.estimator.Estimator(
    training_image,
    "arn:aws:iam::501088278124:role/service-role/AmazonSageMaker-ExecutionRole-20190410T133202", 
    train_instance_count=1, 
    train_instance_type='ml.p3.2xlarge',
    output_path=s3_output_location,
    sagemaker_session=sess
)

### Set some training hyperparameters

Finally, before we train, we provide some additional configuration parameters for the training.

More info here: https://docs.aws.amazon.com/sagemaker/latest/dg/IC-Hyperparameter.html

In [36]:
num_classes=! ls -l {base_dir}/{dataset_name} | wc -l
num_classes=int(num_classes[0]) - 1

num_training_samples=! cat {base_dir}/{dataset_name}_train.lst | wc -l
num_training_samples = int(num_training_samples[0])

# Learn more about the Sagemaker built-in Image Classifier hyperparameters here: https://docs.aws.amazon.com/sagemaker/latest/dg/IC-Hyperparameter.html

# These hyperparameters we won't want to change, as they define things like
# the size of the images we'll be sending for input, the number of training classes we have, etc.
base_hyperparameters=dict(
    use_pretrained_model=1,
    resize=224,
    image_shape='3,224,224',
    num_classes=num_classes,
    num_training_samples=num_training_samples,
)

# These are hyperparameters we may want to tune, as they can affect the model training success:
hyperparameters={
    **base_hyperparameters, 
    **dict(
        learning_rate=0.003851376698585882,
        mini_batch_size=27,
        early_stopping=True,
        early_stopping_min_epochs=6,
        early_stopping_tolerance=.01
    )
}


image_classifier.set_hyperparameters(**hyperparameters)

hyperparameters

{'use_pretrained_model': 1,
 'resize': 224,
 'image_shape': '3,224,224',
 'num_classes': 8,
 'num_training_samples': 1724,
 'learning_rate': 0.003851376698585882,
 'mini_batch_size': 27,
 'early_stopping': True,
 'early_stopping_min_epochs': 6,
 'early_stopping_tolerance': 0.01}

### Start the training
Train our model!

This will take some time because it's provisioning a new container runtime to train our model, then the actual training happens, then the trained model gets uploaded to S3 and the container is shut down.

More info here: https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.Estimator.fit

In [ ]:
%%time

import time
now = str(int(time.time()))
training_job_name = 'ac-7people-new-dataset'

image_classifier.fit(inputs=data_channels, job_name=training_job_name, logs=True)

job = image_classifier.latest_training_job
model_path = f"{base_dir}/{job.name}"

print(f"\n\n Finished training! The model is available for download at: {image_classifier.output_path}/{job.name}/output/model.tar.gz")

2019-07-25 17:56:49 Starting - Starting the training job...
2019-07-25 17:56:59 Starting - Launching requested ML instances......
2019-07-25 17:58:10 Starting - Preparing the instances for training......
2019-07-25 17:59:15 Downloading - Downloading input data
2019-07-25 17:59:15 Training - Downloading the training image............
2019-07-25 18:01:08 Training - Training image download completed. Training in progress.
Docker entrypoint called with argument(s): train
[07/25/2019 18:01:10 INFO 140636498519872] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/image_classification/default-input.json: {u'beta_1': 0.9, u'gamma': 0.9, u'beta_2': 0.999, u'optimizer': u'sgd', u'use_pretrained_model': 0, u'eps': 1e-08, u'epochs': 30, u'lr_scheduler_factor': 0.1, u'num_layers': 152, u'image_shape': u'3,224,224', u'precision_dtype': u'float32', u'mini_batch_size': 32, u'weight_decay': 0.0001, u'learning_rate': 0.1, u'momentum': 0}
[07/25/2019 18:01:10 INFO 14063649851987

## Deploy the trained model
Once a model has been trained, we can use the same `image_classifier` object to create a deployed, fully-managed endpoint.}

More info here: https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.Estimator.deploy

In [39]:
%%time
# Deploying a model to an endpoint takes a few minutes to complete

deployed_endpoint = image_classifier.deploy(
    initial_instance_count = 1,
    instance_type = 'ml.t2.medium'
)

ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateEndpoint operation: The account-level service limit 'Number of instances across active endpoints' is 3 Instances, with current utilization of 3 Instances and a request delta of 1 Instances. Please contact AWS support to request an increase for this limit.

In [19]:
print(deployed_endpoint)

### Clean up

When we're done with the endpoint, we can just delete it and the backing instances will be released.  Run the following cell to delete the endpoint.

In [ ]:
deployed_endpoint.delete_endpoint()

## Calling a deployed endpoint from Python code

If you want to try using a deployed endpoint from Python, here's a function that you can use. It takes in a path to the image you'd like to classify, and a list of all the classes used for training.

In [20]:
import json
import numpy as np
import os

def classify_deployed(file_name, classes):
    payload = None
    with open(file_name, 'rb') as f:
        payload = f.read()
        payload = bytearray(payload)

    deployed_endpoint.content_type = 'application/x-image'
    result = json.loads(deployed_endpoint.predict(payload))
    best_prob_index = np.argmax(result)
    return (classes[best_prob_index], result[best_prob_index])



In [29]:
classify_deployed('c.jpg', ['a', 'b', 'c'])

('c', 0.9948534369468689)

## (Optional) Perform Hyperparameter Tuning

Often, you might not know which values for hyperparameters like `learning_rate` and `mini_batch_size` will yield acceptible results. Traditionally, this meant manually running many training jobs with different hyperparameter values, looking at each trained model's performance, and then picking a winner. 

This type of manual tuning is _very_ time consuming, so you can automate this process using automatic model tuning with SageMaker. Here's some example code to illustrate how to start one of these jobs using the SageMaker Python SDK.

More info here about automatic model tuning: https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html

More info about model tuning for the Image Classification algorithm: https://docs.aws.amazon.com/sagemaker/latest/dg/IC-tuning.html

In [ ]:
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, CategoricalParameter, ContinuousParameter
hyperparameter_ranges = {'optimizer': CategoricalParameter(['sgd', 'adam']),
                         'learning_rate': ContinuousParameter(0.0001, 0.1),
                         'mini_batch_size': IntegerParameter(2, 32),
                        }

objective_metric_name = 'validation:accuracy'

tuner = HyperparameterTuner(image_classifier,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=50,
                            max_parallel_jobs=1)

tuner.fit(inputs=data_channels, logs=True, include_cls_metadata=False)

## Great resources to continue your Deep Learning journey

[3Blue1Brown’s YouTube series on Neural Networks ~ 60 Minutes](https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi)

[Fast.ai’s Practical Deep Learning for Coders ~ 14 Hours](http://www.fast.ai/)
    
[Amazon's Machine Learning University ~ More than 45 hours of courses, videos, and labs](https://aws.amazon.com/training/learning-paths/machine-learning/)
    
[Neural Networks and Deep Learning, by Michael Neilsen ~ 6 Chapter Book](http://neuralnetworksanddeeplearning.com/)

[Amazon SageMaker - Fully-managed Platform](https://aws.amazon.com/sagemaker/)
    
[@gabehollombe's](https://twitter.com/gabehollombe) deep learning tools and demos
- [Jupyter Notebooks](https://github.com/gabehollombe-aws/jupyter-notebooks)
- [Webcam S3 Uploader Tool](https://github.com/gabehollombe-aws/webcam-s3-uploader)
- [SageMaker Inference Web Tool](https://github.com/gabehollombe-aws/webcam-sagemaker-inference)
